In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
from data import *
real_dataset_path='/content/drive/MyDrive/Footprints Datasets/real.npz'
fake_dataset_path='/content/drive/MyDrive/Footprints Datasets/fake.npz'
X_train,Y_train,_,_,X_val,Y_val=Split_Data(real_dataset_path=real_dataset_path,fake_dataset_path=fake_dataset_path)

print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_val))
print(np.shape(Y_val))

(1200, 1997, 101)
(1200, 496)
(300, 1997, 101)
(300, 496)


In [66]:
print("Minimum value",  np.min(X_train))
print("Maximum value", np.max(X_train))
X_train=preprocessing(X_train)
print("Minimum value",  np.min(X_train))
print("Maximum value", np.max(X_train))


print("Minimum value",  np.min(X_val))
print("Maximum value", np.max(X_val))
X_val=preprocessing(X_val)
print("Minimum value",  np.min(X_val))
print("Maximum value", np.max(X_val))

Minimum value 0.0
Maximum value inf
Minimum value 0.0
Maximum value 65500.0
Minimum value 0.0
Maximum value inf
Minimum value 0.0
Maximum value 65500.0


In [113]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(Y_train), Y_train)
print(class_weights)

{0: 0.2, 1: 0.8}


In [67]:
# Model
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.initializers import RandomUniform
import tensorflow as tf

In [68]:
# GRADED FUNCTION: model

def model(input_shape=(1997,101)):
    """
    Function creating the model's graph in Keras.

    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """

    X_input = Input(shape = input_shape)

    ### START CODE HERE ###

    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=17, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)

    #   # Print output shape after Conv1D
    # print("Output shape after Conv1D:", X.shape)

    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization

    # # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.8)(X)                                  # dropout (use 0.8)

    # # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)

    return model

In [69]:
model_eagle = model()
model_eagle.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1997, 101)]       0         
                                                                 
 conv1d_6 (Conv1D)           (None, 496, 196)          336728    
                                                                 
 batch_normalization_18 (Ba  (None, 496, 196)          784       
 tchNormalization)                                               
                                                                 
 activation_6 (Activation)   (None, 496, 196)          0         
                                                                 
 dropout_24 (Dropout)        (None, 496, 196)          0         
                                                                 
 gru_12 (GRU)                (None, 496, 128)          125184    
                                                           

In [70]:
zeros = np.count_nonzero(Y_train == 0)
ones = np.count_nonzero(Y_train == 1)
print(f"Real Data: 0s: {zeros}, 1s:{ones}")
print(zeros/ones)

Real Data: 0s: 535701, 1s:59499
9.003529471083548


In [71]:
class_weights = {0: 1.0, 1: 10.0}  # Example class weights
@keras.saving.register_keras_serializable()
# Define weighted binary cross-entropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    # Clip predicted values to prevent log(0) and log(1) cases
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

    # Compute weighted binary cross-entropy
    weights = tf.constant([class_weights[i] for i in range(len(class_weights))])
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    return tf.reduce_mean(bce(y_true, y_pred) * weights)

In [72]:
# opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_eagle.compile(loss=weighted_binary_crossentropy, optimizer=opt, metrics=["accuracy"])

In [73]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)

240/240 [==============================] - 14s 42ms/step - loss: 3.4101 - accuracy: 0.7822


In [74]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=30)

Epoch 1/30
75/75 [==============================] - 6s 48ms/step - loss: 1.9000 - accuracy: 0.8957
Epoch 2/30
75/75 [==============================] - 4s 47ms/step - loss: 1.8285 - accuracy: 0.8981
Epoch 3/30
75/75 [==============================] - 4s 54ms/step - loss: 1.8075 - accuracy: 0.8989
Epoch 4/30
75/75 [==============================] - 3s 47ms/step - loss: 1.7998 - accuracy: 0.8992
Epoch 5/30
75/75 [==============================] - 4s 47ms/step - loss: 1.7926 - accuracy: 0.8994
Epoch 6/30
75/75 [==============================] - 4s 52ms/step - loss: 1.7889 - accuracy: 0.8996
Epoch 7/30
75/75 [==============================] - 4s 49ms/step - loss: 1.7903 - accuracy: 0.8996
Epoch 8/30
75/75 [==============================] - 4s 47ms/step - loss: 1.7837 - accuracy: 0.8997
Epoch 9/30
75/75 [==============================] - 4s 47ms/step - loss: 1.7799 - accuracy: 0.8998
Epoch 10/30
75/75 [==============================] - 4s 55ms/step - loss: 1.7806 - accuracy: 0.8998
Epoch 11/

In [75]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=10)

Epoch 1/10
75/75 [==============================] - 4s 48ms/step - loss: 0.6857 - accuracy: 0.9455
Epoch 2/10
75/75 [==============================] - 4s 54ms/step - loss: 0.6733 - accuracy: 0.9493
Epoch 3/10
75/75 [==============================] - 3s 46ms/step - loss: 0.6974 - accuracy: 0.9449
Epoch 4/10
75/75 [==============================] - 4s 47ms/step - loss: 0.6392 - accuracy: 0.9526
Epoch 5/10
75/75 [==============================] - 4s 48ms/step - loss: 0.5661 - accuracy: 0.9565
Epoch 6/10
75/75 [==============================] - 4s 53ms/step - loss: 0.5614 - accuracy: 0.9584
Epoch 7/10
75/75 [==============================] - 4s 47ms/step - loss: 0.5319 - accuracy: 0.9606
Epoch 8/10
75/75 [==============================] - 4s 47ms/step - loss: 0.5600 - accuracy: 0.9585
Epoch 9/10
75/75 [==============================] - 4s 52ms/step - loss: 0.4989 - accuracy: 0.9629
Epoch 10/10
75/75 [==============================] - 4s 49ms/step - loss: 0.5514 - accuracy: 0.9593


In [76]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=10)

Epoch 1/10
75/75 [==============================] - 4s 48ms/step - loss: 0.5015 - accuracy: 0.9633
Epoch 2/10
75/75 [==============================] - 4s 54ms/step - loss: 0.4723 - accuracy: 0.9657
Epoch 3/10
75/75 [==============================] - 4s 47ms/step - loss: 0.4319 - accuracy: 0.9692
Epoch 4/10
75/75 [==============================] - 4s 47ms/step - loss: 0.4542 - accuracy: 0.9682
Epoch 5/10
75/75 [==============================] - 4s 51ms/step - loss: 0.4635 - accuracy: 0.9674
Epoch 6/10
75/75 [==============================] - 4s 51ms/step - loss: 0.4174 - accuracy: 0.9704
Epoch 7/10
75/75 [==============================] - 4s 47ms/step - loss: 0.3931 - accuracy: 0.9725
Epoch 8/10
75/75 [==============================] - 4s 48ms/step - loss: 0.4317 - accuracy: 0.9701
Epoch 9/10
75/75 [==============================] - 4s 55ms/step - loss: 0.4292 - accuracy: 0.9700
Epoch 10/10
75/75 [==============================] - 3s 47ms/step - loss: 0.4760 - accuracy: 0.9676


In [59]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)

240/240 [==============================] - 142s 573ms/step - loss: 0.9543 - accuracy: 0.5938


In [77]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)

240/240 [==============================] - 141s 589ms/step - loss: 0.7612 - accuracy: 0.7689


In [81]:
model_eagle.fit(X_train,Y_train, batch_size = 64, epochs=1)

19/19 [==============================] - 71s 3s/step - loss: 0.6902 - accuracy: 0.8644


In [77]:
model_eagle.save('/content/model_1.keras')

In [79]:
loss, acc = model_eagle.evaluate(X_val, Y_val)
print(loss)
print(acc)

10/10 [==============================] - 0s 43ms/step - loss: 0.2799 - accuracy: 0.9796
0.27992844581604004
0.9796304106712341


In [80]:
predictions = model_eagle.predict(X_val)
print(np.shape(predictions))

10/10 [==============================] - 1s 37ms/step
(300, 496, 1)


In [81]:
print(predictions[0,:].T)
print(np.max(predictions[0,:]))

# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[0,:] >= 0.5).astype(int)
print(binary_labels.T)

[[9.79759991e-02 4.11665998e-02 2.04048753e-02 1.16588390e-02
  7.34589295e-03 4.88874502e-03 3.31032928e-03 2.21129600e-03
  1.42732123e-03 8.89947347e-04 5.54073660e-04 3.66203400e-04
  2.70630320e-04 2.25537369e-04 2.06012715e-04 1.98628200e-04
  1.96395456e-04 1.95884350e-04 1.95656234e-04 1.95303204e-04
  1.94825945e-04 1.94332621e-04 1.93928747e-04 1.93674670e-04
  1.93572749e-04 1.93603395e-04 1.93718050e-04 1.93893778e-04
  1.94093940e-04 1.94315580e-04 1.94533772e-04 1.94752967e-04
  1.94976295e-04 1.95201923e-04 1.95411252e-04 1.95605491e-04
  1.95781278e-04 1.95945046e-04 1.96111403e-04 1.96284993e-04
  1.96437410e-04 1.96574154e-04 1.96692665e-04 1.96844077e-04
  1.96966866e-04 1.97019443e-04 1.97064903e-04 1.97155707e-04
  1.97268528e-04 1.97407193e-04 1.97504734e-04 1.97561414e-04
  1.97589296e-04 1.97652218e-04 1.97780057e-04 1.97852278e-04
  1.97885107e-04 1.97883041e-04 1.97938323e-04 1.98003821e-04
  1.98072754e-04 1.98144335e-04 1.98201786e-04 1.98236376e-04
  1.9827

In [82]:
print(Y_val[0])
print(np.max(Y_val[0]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [83]:
print(predictions[1].T)
print(np.max(predictions[1]))

# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[1,:] >= 0.5).astype(int)
print(binary_labels.T)

[[9.7975999e-02 4.1166607e-02 2.0404875e-02 1.1658839e-02 7.3458930e-03
  4.8887450e-03 3.3103267e-03 2.2112823e-03 1.4272743e-03 8.8991004e-04
  5.5406045e-04 3.6620360e-04 2.7062825e-04 2.2553437e-04 2.0600896e-04
  1.9862063e-04 1.9638235e-04 1.9586658e-04 1.9564168e-04 1.9528181e-04
  1.9479661e-04 1.9430985e-04 1.9391951e-04 1.9367263e-04 1.9356649e-04
  1.9357498e-04 1.9367042e-04 1.9383040e-04 1.9402528e-04 1.9424113e-04
  1.9446108e-04 1.9467017e-04 1.9486812e-04 1.9505345e-04 1.9522074e-04
  1.9540901e-04 1.9561202e-04 1.9581561e-04 1.9601568e-04 1.9620343e-04
  1.9636961e-04 1.9653051e-04 1.9667747e-04 1.9681497e-04 1.9692742e-04
  1.9700835e-04 1.9708145e-04 1.9714273e-04 1.9722413e-04 1.9732081e-04
  1.9742431e-04 1.9752544e-04 1.9761473e-04 1.9767598e-04 1.9772745e-04
  1.9777892e-04 1.9784021e-04 1.9790491e-04 1.9797380e-04 1.9803537e-04
  1.9808502e-04 1.9812393e-04 1.9816929e-04 1.9823202e-04 1.9827209e-04
  1.9831615e-04 1.9836740e-04 1.9841772e-04 1.9846503e-04 1.9851

In [84]:
print(Y_val[1])
print(np.max(Y_val[1]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [162]:
loss, acc = model_eagle.evaluate(np.expand_dims(X_val[1,:,:], axis=0),np.expand_dims(Y_val[1,:], axis=0))

1/1 [==============================] - 0s 79ms/step - loss: 4.0995 - accuracy: 0.7198


In [85]:
print(predictions[50].T)
print(np.max(predictions[50]))
# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[50,:] >= 0.5).astype(int)
print(binary_labels.T)

[[9.79759991e-02 4.11665998e-02 2.04048753e-02 1.16588390e-02
  7.34589295e-03 4.88874502e-03 3.31032672e-03 2.21128645e-03
  1.42728782e-03 8.89920106e-04 5.54063066e-04 3.66202876e-04
  2.70629302e-04 2.25536947e-04 2.06013297e-04 1.98628011e-04
  1.96391702e-04 1.95879664e-04 1.95651941e-04 1.95292057e-04
  1.94807930e-04 1.94317239e-04 1.93922446e-04 1.93673375e-04
  1.93572196e-04 1.93595260e-04 1.93705884e-04 1.93875501e-04
  1.94076740e-04 1.94295950e-04 1.94518580e-04 1.94741267e-04
  1.94961060e-04 1.95175322e-04 1.95377157e-04 1.95566710e-04
  1.95744680e-04 1.95913104e-04 1.96074194e-04 1.96231093e-04
  1.96376175e-04 1.96511406e-04 1.96636232e-04 1.96765453e-04
  1.96881054e-04 1.96971538e-04 1.97055881e-04 1.97150061e-04
  1.97248024e-04 1.97349043e-04 1.97435802e-04 1.97508474e-04
  1.97570247e-04 1.97639209e-04 1.97724046e-04 1.97790796e-04
  1.97843212e-04 1.97883594e-04 1.97938905e-04 1.97995148e-04
  1.98051232e-04 1.98106558e-04 1.98156617e-04 1.98199326e-04
  1.9824

In [86]:
print(Y_val[50])
print(np.max(Y_val[50]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [165]:
print(predictions[50].T)
print(np.max(predictions[50]))

print(Y_val[50])
print(np.max(Y_val[50]))

[[9.3152868e-03 9.5999995e-03 9.1443863e-03 8.5982513e-03 8.2309814e-03
  8.0632390e-03 8.0288537e-03 8.0559533e-03 8.0954954e-03 8.1244325e-03
  8.1381183e-03 8.1406645e-03 8.1379153e-03 8.1341285e-03 8.1313392e-03
  8.1299441e-03 8.1295818e-03 8.1297364e-03 8.1300242e-03 8.1302440e-03
  8.1303595e-03 8.1303865e-03 8.1303669e-03 8.1303446e-03 8.1303213e-03
  8.1303138e-03 8.1303101e-03 8.1303054e-03 8.1303101e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03
  8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303138e-03 8.1303

In [76]:
np.max(predictions)

0.7341588

In [87]:
import keras
from keras.models import load_model
loaded_model = keras.models.load_model('/content/model_1.keras')

In [88]:
loss, acc = loaded_model.evaluate(X_val, Y_val)
print(loss)
print(acc)

10/10 [==============================] - 2s 71ms/step - loss: 0.2799 - accuracy: 0.9796
0.27992844581604004
0.9796304106712341


In [62]:
# rand_model = model()

# opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# rand_model.compile(loss=weighted_binary_crossentropy, optimizer=opt, metrics=["accuracy"])
# loss, acc = x.evaluate(X_val, Y_val)
